In [2]:
# Import required libraries
import boto3 #Video files get read through this
import cv2
import os
import pandas as pd
import math
import numpy as np
import random
import io
import s3fs # Parquet files get read through this
import zlib # For compression
import time # To calculate download time

In [3]:
#!pip install moviepy
#!pip install cv2
#!pip3 install opencv-python-headless
#!pip install h5py
#!pip install tensorflow

In [4]:
import configparser

config = configparser.ConfigParser()
config.read('/home/ec2-user/.aws/credentials')

['/home/ec2-user/.aws/credentials']

In [5]:
# AWS credentials and S3 settings
aws_access_key_id = config["root"]['aws_access_key_id']
aws_secret_access_key = config["root"]['aws_secret_access_key']
bucket_name = 'asl-capstone'
prefix = 'youtube-asl/test_sample/'
save_path = '/content/temp_folder'
s3_URI = 's3://asl-capstone/'

In [6]:
# Initialize a Boto3 S3 client
s3 = boto3.client(
    's3',
    aws_access_key_id=aws_access_key_id,
    aws_secret_access_key=aws_secret_access_key,
    region_name='us-west-2'
)

In [7]:
import s3fs
fs = s3fs.S3FileSystem(anon=True)

In [8]:
import boto3
import botocore
import time
import io
import numpy as np

# AWS 
aws_access_key_id = config["root"]['aws_access_key_id']
aws_secret_access_key = config["root"]['aws_secret_access_key']
bucket_name = 'asl-capstone'
prefix = 'youtube-asl/test_sample/'
s3_URI = 's3://asl-capstone/'

s3_file_key = prefix + 'numpy_files/RGB/padded_joined_videos_150.npy'

s3 = boto3.client('s3', aws_access_key_id=aws_access_key_id, aws_secret_access_key=aws_secret_access_key)

start_time = time.time()

try:
    response = s3.get_object(Bucket=bucket_name, Key=s3_file_key)
    data = response['Body'].read()
    
    loaded_array = np.load(io.BytesIO(data))

    end_time = time.time()
    elapsed_time = end_time - start_time
    print(f"Time to retrieve file from S3: {elapsed_time} seconds")

except botocore.exceptions.ClientError as e:
    if e.response['Error']['Code'] == "NoSuchKey":
        print("The specified key does not exist.")
    else:
        print("An error occurred: ", e)


Time to retrieve file from S3: 1014.574492931366 seconds


In [10]:
import numpy as np
import time

# Define the output .npy file path
npy_file_path = 'padded_joined_videos_150.npy'

try:
    start_time = time.time()

    # Save the loaded_array as a .npy file
    np.save(npy_file_path, loaded_array)

    end_time = time.time()
    elapsed_time = end_time - start_time
    print(f"Time to save loaded_array as a .npy file: {elapsed_time} seconds")

    # Print the location of the .npy file
    print(f"File saved at: {os.path.abspath(npy_file_path)}")
except Exception as e:
    print(f"An error occurred: {e}")

An error occurred: Not enough free space to write 9437184000 bytes


In [11]:
import h5py
import os
import time

hdf5_file_path = 'padded_joined_videos_150.h5'

try:
    start_time = time.time()

    # Create an HDF5 file and store the loaded_array in it
    with h5py.File(hdf5_file_path, 'w') as hf:
        hf.create_dataset('data', data=loaded_array)

    end_time = time.time()
    elapsed_time = end_time - start_time
    print(f"Time to store loaded_array in HDF5 file: {elapsed_time} seconds")
    
    # Print the location of the HDF5 file
    print(f"File saved at: {os.path.abspath(hdf5_file_path)}")
except Exception as e:
    print(f"An error occurred: {e}")


Time to store loaded_array in HDF5 file: 53.60840559005737 seconds
File saved at: /home/ec2-user/ASL-Translator/data_processing/padded_joined_videos_150.h5


In [11]:
import boto3
import os
import time

# AWS credentials and S3 settings
s3_folder = "size_test"  

# Path to the local HDF5 file
local_file_path = "padded_joined_videos_150.h5"

# Initialize the S3 client
s3 = boto3.client('s3', aws_access_key_id=aws_access_key_id, aws_secret_access_key=aws_secret_access_key)

start_time = time.time()

# Upload the file to S3 in the specified folder
s3_object_key = f"{s3_folder}/{os.path.basename(local_file_path)}"
s3.upload_file(local_file_path, bucket_name, s3_object_key)

end_time = time.time()
elapsed_time = end_time - start_time
print(f"Time to upload file to S3: {elapsed_time} seconds")
print(f"File uploaded to S3: s3://{bucket_name}/{s3_object_key}")

Time to upload file to S3: 90.45855116844177 seconds
File uploaded to S3: s3://asl-capstone/size_test/padded_joined_videos_150.h5


In [7]:
import boto3
import h5py
import time
import io

# AWS credentials and S3 settings
s3_folder = "size_test"  
hdf5_file_name = "padded_joined_videos_150.h5"  

# Initialize the S3 client
s3 = boto3.client('s3', aws_access_key_id=aws_access_key_id, aws_secret_access_key=aws_secret_access_key)

start_time = time.time()

# Download the HDF5 file from S3 and load it directly into memory
s3_object_key = f"{s3_folder}/{hdf5_file_name}"
response = s3.get_object(Bucket=bucket_name, Key=s3_object_key)
data = response['Body'].read()

end_time_download = time.time()
elapsed_time_download = end_time_download - start_time
print(f"Time to download and load HDF5 data from S3: {elapsed_time_download} seconds")

Time to download and load HDF5 data from S3: 493.39018154144287 seconds


In [ ]:
import tensorflow as tf

# Convert loaded_array to a TensorFlow tensor
loaded_tensor = tf.constant(loaded_array)

2023-11-08 08:46:57.055131: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2023-11-08 08:46:57.959708: E tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:9342] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2023-11-08 08:46:57.959796: E tensorflow/compiler/xla/stream_executor/cuda/cuda_fft.cc:609] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2023-11-08 08:46:57.968116: E tensorflow/compiler/xla/stream_executor/cuda/cuda_blas.cc:1518] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2023-11-08 08:46:58.409214: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2023-11-08 08:46:58.411966: I tensorflow/core/platform/cpu_feature_guard.cc:182] This Tens

In [ ]:
import tensorflow as tf
import os
import time

# Convert loaded_array to a TensorFlow tensor
loaded_tensor = tf.constant(loaded_array)

# Define the output TensorFlow file path
tf_file_path = 'padded_joined_videos_150.tf'

try:
    start_time = time.time()

    # Save the TensorFlow tensor as a TensorFlow file
    tf.io.write_file(tf_file_path, loaded_tensor.numpy())

    end_time = time.time()
    elapsed_time = end_time - start_time
    print(f"Time to store loaded_array as a TensorFlow file: {elapsed_time} seconds")
    
    # Print the location of the TensorFlow file
    print(f"File saved at: {os.path.abspath(tf_file_path)}")
except Exception as e:
    print(f"An error occurred: {e}")

2023-11-08 08:25:21.153546: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2023-11-08 08:25:22.076725: E tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:9342] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2023-11-08 08:25:22.076821: E tensorflow/compiler/xla/stream_executor/cuda/cuda_fft.cc:609] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2023-11-08 08:25:22.082138: E tensorflow/compiler/xla/stream_executor/cuda/cuda_blas.cc:1518] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2023-11-08 08:25:22.548392: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2023-11-08 08:25:22.550805: I tensorflow/core/platform/cpu_feature_guard.cc:182] This Tens